In [1]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dense, Dropout, 
                                     Flatten, BatchNormalization)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


In [2]:
# Image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Define image augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No augmentation for validation and test data — only rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Path to CSV files
train_csv = '../split_data/training_data.csv'
val_csv = '../split_data/validation_data.csv'
test_csv = '../split_data/testing_data.csv'

# Path to image folders
train_dir = '../split_data/training'
val_dir = '../split_data/validation'
test_dir = '../split_data/testing'

# Load training data
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)
test_df = pd.read_csv(test_csv)

# Create data generators using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='img_id',
    y_col='class_id',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=val_dir,
    x_col='img_id',
    y_col='class_id',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='img_id',
    y_col='class_id',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 8442 validated image filenames belonging to 3 classes.
Found 2814 validated image filenames belonging to 3 classes.
Found 2814 validated image filenames belonging to 3 classes.


In [3]:
from sklearn.utils.class_weight import compute_class_weight


class_labels = train_df['class_id']
classes = np.unique(class_labels)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=class_labels
)
class_weights_dict = dict(zip(classes, class_weights))
print("Computed class weights:", class_weights_dict)


Computed class weights: {'drinking': np.float64(2.0172043010752687), 'safe_driving': np.float64(1.884795713328868), 'using_phone': np.float64(0.5066618653222903)}


In [4]:

def build_stable_custom_cnn(input_shape=(224, 224, 3), num_classes=3):
    model = Sequential()

    # Block 1
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                     kernel_regularizer=l2(0.0001), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Block 2
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu',
                     kernel_regularizer=l2(0.0001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Block 3
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu',
                     kernel_regularizer=l2(0.0001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # FC Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(Dropout(0.4))  # Only one dropout here
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile
model = build_stable_custom_cnn()

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

model.summary()


C:\Users\nisha\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,715 (49.36 MB)

 Trainable params: 12,939,267 (49.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
# Early stopping & learning rate decay
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6, verbose=1)

# Save the best model
checkpoint = ModelCheckpoint("best_stable_model.h5", monitor='val_loss', save_best_only=True, verbose=1)



In [6]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)


C:\Users\nisha\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 963ms/step - accuracy: 0.6126 - loss: 2.9574 - precision: 0.5486 - recall: 0.1745
Epoch 1: val_loss improved from inf to 6.86923, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 278s 1s/step - accuracy: 0.6127 - loss: 2.9519 - precision: 0.5489 - recall: 0.1744 - val_accuracy: 0.5821 - val_loss: 6.8692 - val_precision: 0.5821 - val_recall: 0.5821 - learning_rate: 5.0000e-04
Epoch 2/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 952ms/step - accuracy: 0.6615 - loss: 0.9778 - precision: 0.6776 - recall: 0.3707
Epoch 2: val_loss improved from 6.86923 to 0.93269, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 271s 1s/step - accuracy: 0.6614 - loss: 0.9778 - precision: 0.6777 - recall: 0.3706 - val_accuracy: 0.6521 - val_loss: 0.9327 - val_precision: 0.7831 - val_recall: 0.4851 - learning_rate: 5.0000e-04
Epoch 3/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 969ms/step - accuracy: 0.6518 - loss: 0.9402 - precision: 0.7169 - recall: 0.3487
Epoch 3: val_loss improved from 0.93269 to 0.80249, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 275s 1s/step - accuracy: 0.6518 - loss: 0.9402 - precision: 0.7169 - recall: 0.3487 - val_accuracy: 0.6578 - val_loss: 0.8025 - val_precision: 0.7357 - val_recall: 0.6439 - learning_rate: 5.0000e-04
Epoch 4/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6510 - loss: 0.9128 - precision: 0.7625 - recall: 0.3193
Epoch 4: val_loss did not improve from 0.80249
264/264 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - accuracy: 0.6511 - loss: 0.9128 - precision: 0.7626 - recall: 0.3194 - val_accuracy: 0.6578 - val_loss: 0.9100 - val_precision: 0.6633 - val_recall: 0.6567 - learning_rate: 5.0000e-04
Epoch 5/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6515 - loss: 0.9055 - precision: 0.7642 - recall: 0.3120
Epoch 5: val_loss improved from 0.80249 to 0.78333, saving model to best_stable_model.h5


...

264/264 ━━━━━━━━━━━━━━━━━━━━ 255s 967ms/step - accuracy: 0.6585 - loss: 0.7511 - precision: 0.6585 - recall: 0.6585 - val_accuracy: 0.6578 - val_loss: 0.5989 - val_precision: 0.6578 - val_recall: 0.6578 - learning_rate: 1.5000e-04
Epoch 18/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 922ms/step - accuracy: 0.6549 - loss: 0.7486 - precision: 0.7760 - recall: 0.4820
Epoch 18: val_loss did not improve from 0.59888
264/264 ━━━━━━━━━━━━━━━━━━━━ 259s 982ms/step - accuracy: 0.6550 - loss: 0.7486 - precision: 0.7763 - recall: 0.4817 - val_accuracy: 0.6578 - val_loss: 0.6318 - val_precision: 0.9350 - val_recall: 0.6134 - learning_rate: 1.5000e-04
Epoch 19/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 941ms/step - accuracy: 0.6564 - loss: 0.7457 - precision: 0.9342 - recall: 0.3721
Epoch 19: val_loss improved from 0.59888 to 0.56835, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.6564 - loss: 0.7457 - precision: 0.9342 - recall: 0.3721 - val_accuracy: 0.6578 - val_loss: 0.5684 - val_precision: 0.9153 - val_recall: 0.6489 - learning_rate: 1.5000e-04
Epoch 20/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 938ms/step - accuracy: 0.6595 - loss: 0.7338 - precision: 0.9384 - recall: 0.3784
Epoch 20: val_loss did not improve from 0.56835
264/264 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.6594 - loss: 0.7338 - precision: 0.9384 - recall: 0.3784 - val_accuracy: 0.6578 - val_loss: 0.6394 - val_precision: 0.9927 - val_recall: 0.5810 - learning_rate: 1.5000e-04
Epoch 21/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 902ms/step - accuracy: 0.6547 - loss: 0.7424 - precision: 0.9491 - recall: 0.3638
Epoch 21: val_loss did not improve from 0.56835
264/264 ━━━━━━━━━━━━━━━━━━━━ 255s 966ms/step - accuracy: 0.6547 - loss: 0.7423 - precision: 0.9491 - recall: 0.3638 - val_accuracy: 0.6578 - val_loss: 0.5704 - val_precision: 0.9428 - val_recall: 

264/264 ━━━━━━━━━━━━━━━━━━━━ 255s 965ms/step - accuracy: 0.6751 - loss: 0.7166 - precision: 0.9253 - recall: 0.3852 - val_accuracy: 0.7534 - val_loss: 0.5051 - val_precision: 0.9725 - val_recall: 0.6912 - learning_rate: 4.5000e-05
Epoch 24/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 923ms/step - accuracy: 0.7083 - loss: 0.6571 - precision: 0.9170 - recall: 0.4479
Epoch 24: val_loss improved from 0.50514 to 0.46820, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 260s 984ms/step - accuracy: 0.7083 - loss: 0.6571 - precision: 0.9170 - recall: 0.4480 - val_accuracy: 0.7846 - val_loss: 0.4682 - val_precision: 0.9830 - val_recall: 0.7175 - learning_rate: 4.5000e-05
Epoch 25/25
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 943ms/step - accuracy: 0.7294 - loss: 0.6194 - precision: 0.9167 - recall: 0.4739
Epoch 25: val_loss improved from 0.46820 to 0.41385, saving model to best_stable_model.h5


264/264 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.7294 - loss: 0.6194 - precision: 0.9167 - recall: 0.4739 - val_accuracy: 0.7985 - val_loss: 0.4138 - val_precision: 0.9632 - val_recall: 0.7726 - learning_rate: 4.5000e-05
Restoring model weights from the end of the best epoch: 25.


Worked